<a href="https://colab.research.google.com/github/abhinivesh-s/Semi-Supervised-Text/blob/main/sklearn_algos_on_Enron_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
import torch.utils.data as Data
import pickle
# TensorFlow and tf.keras
import tensorflow as tf
import re
import email
from sklearn import model_selection

In [ ]:
print(np.__version__)
import sklearn
print(sklearn.__version__)

1.16.2
0.24.2


In [ ]:
!pip install numpy==1.16.2
import numpy as np

     |████████████████████████████████| 17.3 MB 64 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 0.18.2 requires numpy>=1.17, but you have numpy 1.16.2 which is incompatible.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.16.2 which is incompatible.
pyerfa 2.0.0 requires numpy>=1.17, but you have numpy 1.16.2 which is incompatible.
pyarrow 3.0.0 requires numpy>=1.16.6, but you have numpy 1.16.2 which is incompatible.
kapre 0.3.5 requires numpy>=1.18.5, but you have numpy 1.16.2 which is incompatible.
jaxlib 0.1.69+cuda110 requires numpy>=1.17, but you have numpy 1.16.2 which is incompatible.
jax 0.2.17 requires numpy>=1.17, but you have numpy 1.16.2 which is incompatible.
datascience

In [ ]:
!pip install --upgrade scikit-learn  # Do this to use sklearn SelfTrainingClassifier

In [ ]:
%cd "/content/drive/My Drive/Colab Notebooks/MixText-master"
!ls

/content/drive/My Drive/Colab Notebooks/MixText-master
 bert_enron.xlsx
 bert_imdb_2k.xlsx
 bert-sklearn
 bert_sklearn.log
 code
 data
 data.pkl
 data_temp
 enron_mlp.xlsx
 enron_sgd.xlsx
 glove.6B.100d.txt
 glove.6B.200d.txt
 glove.6B.300d.txt
 glove.6B.50d.txt
 glove.6B.zip
 glove.840B.300d.txt
 glove.840B.300d.zip
 LICENSE
 lstm_imdb.xlsx
 model.png
 nb_imdb.xlsx
 nb_ng.xlsx
 README.md
 rf_imdb.xlsx
 rf_ng.xlsx
'Running MixText v3.ipynb'
'Running MixText v5 ISIS Data.ipynb'
'Semi-Supervised sklearn v2.ipynb'
 sgd_imdb.xlsx
 sgd_ng_218_posts.xlsx
 sgd_ng.xlsx
 tf-cnn_enron_GloVE_840.xlsx
 tf-cnn_enron_GloVE_non-trainable_840.xlsx
 tf-cnn_imdb_GloVE_840.xlsx
 tf-cnn_imdb_GloVE_non-trainable_840.xlsx
 tf-cnn_ng_GloVE_840.xlsx
 tf-cnn_ng_GloVE_non-trainable_840_218_posts.xlsx
 tf-cnn_ng_GloVE_non-trainable_840.xlsx
 tf-lstm_enron_GloVE_840.xlsx
 tf-lstm_enron_GloVE.xlsx
 tf-lstm_imdb_GloVE_840.xlsx
 tf-lstm_imdb_GloVE.xlsx
 tf-lstm_ng_GloVE_840_non-trainable_840_218_posts.xlsx
 tf-lstm_

In [ ]:
import os
os.listdir('data/enron/to_use_for_experiments')

['Original categories by UCB students.txt',
 'setup1_legal.csv',
 'setup2_non_emotional.csv',
 'setup1_company_or_business_related.csv',
 'setup2_emotional.csv',
 'setup1_personal.csv',
 'setup1_logistics.csv',
 'setup2_confidential.csv',
 'setup1_multiclass.csv',
 'setup2_multiclass.csv',
 'ReadMe.txt',
 'labeled']

In [ ]:
setup_1 = [i for i in os.listdir('data/enron/to_use_for_experiments') if ('setup1_' in i) and ('setup1_m' not in i)]
setup_2 = [i for i in os.listdir('data/enron/to_use_for_experiments') if ('setup2_' in i) and ('setup2_m' not in i)]

# SGD

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

In [ ]:
# Parameters
sdg_params = dict(alpha=1e-5, penalty='l2', loss='log', random_state=0)
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

#SGD
def eval_and_print_metrics_df(clf, X_train, y_train, X_test, y_test, thresh = None, kbest = None, label = None):

    dict1 = {}


    print("Number of training samples:", len(X_train))
    print("Unlabeled samples in training set:",
          sum(1 for x in y_train if x == -1))
    
    dict1['Labeled'] = len(X_train) - sum(1 for x in y_train if x == -1)
    dict1['UnLabeled'] = sum(1 for x in y_train if x == -1)
    dict1['label_used'] = label

    tr_0, tr_1 = sum(1 for x in y_train if x == 0), sum(1 for x in y_train if x == 1)
    te_0, te_1 = sum(1 for x in y_test if x == 0), sum(1 for x in y_test if x == 1)

    dict1['Train 0-1-trivial'] = str(tr_0)+'-'+str(tr_1)+'-'+\
                                                str(max(tr_0, tr_1)/(tr_0+tr_1))

    dict1['Test 0-1-trivial'] = str(te_0)+'-'+str(te_1)+'-'+\
                                                str(max(te_0, te_1)/(te_0+te_1))

    
    # if sum(1 for x in y_train if x == -1) == 0:
    #     dict1['type'] = 'Supervised'
    # else:
    #     dict1['type'] = 'Semi-Supervised'

    dict1['Threshold'] = thresh
    dict1['K-Best'] = kbest

    # print('y_train:', y_train)
    # with tf.device('/device:GPU:0'):
    clf.fit(X_train, y_train)
    y_pred = np.array(clf.predict(X_test))
      
    print("Micro-averaged F1 score on test set: "
          "%0.3f" % f1_score(y_test, y_pred, average='micro'))
    print("Accuracy Score: ", accuracy_score(y_test, y_pred))

    dict1['Accuracy'] = accuracy_score(y_test, y_pred)
    dict1['roc-auc'] = roc_auc_score(y_test, y_pred)
    dict1['precision'] = precision_score(y_test, y_pred)
    dict1['recall'] = recall_score(y_test, y_pred)
    dict1['f1'] = f1_score(y_test, y_pred)

    print("-" * 10)
    print()

    return dict1

In [ ]:
# SGD - Log Loss(Logistic regression).
if __name__ == "__main__":



    threshold=[0.4, 0.5, 0.6, 0.7, 0.8]
    df_sgd_en = pd.DataFrame()

    path = 'data/enron/main'
    main_labels = ['message', 'subject', 'body', 'full_message']

    for i in setup_1 + setup_2:      
      j = i[:-4]
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Enron DATA - ", j, "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

      X_train, X_test, y_train, y_test = pd.read_csv(os.path.join(path, j+'_train.csv'))[main_labels],\
            pd.read_csv(os.path.join(path, j+'_test.csv'))[main_labels], pd.read_csv(os.path.join(path, j+'_train.csv'))[j],\
            pd.read_csv(os.path.join(path, j+'_test.csv'))[j]

      
      X_u = pd.read_csv(os.path.join(path, 'enron_unlabeled_2k.csv'))
      y_u = pd.Series(np.array([-1 for i in range(X_u.shape[0])]))


      to_be_used = 'full_message'
      X_train = X_train[to_be_used]
      X_test = X_test[to_be_used]
      X_u = X_u[to_be_used]



      print("Supervised SGDClassifier on the training data:")

      pipeline = Pipeline([
          ('vect', CountVectorizer(**vectorizer_params)),
          ('tfidf', TfidfTransformer()),
          ('clf', SGDClassifier(**sdg_params)),
      ])
          
      temp = eval_and_print_metrics_df(pipeline, X_train, y_train, X_test, y_test, thresh = None, kbest = None, label = j)
      df_sgd_en = df_sgd_en.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on the training data (rest "
            "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(SGDClassifier(**sdg_params), threshold = t, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, pd.concat([X_train, X_u]), pd.concat([y_train, y_u]), X_test, y_test, thresh = t, kbest = None, label=j)
        df_sgd_en = df_sgd_en.append(temp, ignore_index=True)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Enron DATA -  setup1_legal ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised SGDClassifier on the training data:
Number of training samples: 297
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.882
Accuracy Score:  0.8819875776397516
----------

SelfTrainingClassifier on the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 2297
Unlabeled samples in training set: 2000
End of iteration 1, added 2000 new labels.
Micro-averaged F1 score on test set: 0.901
Accuracy Score:  0.9006211180124224
----------

---------------------------------Threshold =  0.5 ---------------------------------
Number of training samples: 2297
Unlabeled samples in training set: 2000
End of iteration 1, added 2000 new labels.
Micro-averaged F1 score on test set: 0.901
Accuracy Score:  0.9006211180124224
----------

---------------------------------Threshold =  0.6 --

In [ ]:
df_sgd_en

,Accuracy,K-Best,Labeled,Test 0-1-trivial,Threshold,Train 0-1-trivial,UnLabeled,f1,label_used,precision,recall,roc-auc
0,0.881988,None,297.0,144-17-0.8944099378881988,None,265-32-0.8922558922558923,0.0,0.240000,setup1_legal,0.375000,0.176471,0.570874
1,0.900621,None,297.0,144-17-0.8944099378881988,0.4,265-32-0.8922558922558923,2000.0,0.272727,setup1_legal,0.600000,0.176471,0.581291
2,0.900621,None,297.0,144-17-0.8944099378881988,0.5,265-32-0.8922558922558923,2000.0,0.272727,setup1_legal,0.600000,0.176471,0.581291
3,0.900621,None,297.0,144-17-0.8944099378881988,0.6,265-32-0.8922558922558923,2000.0,0.272727,setup1_legal,0.600000,0.176471,0.581291
4,0.900621,None,297.0,144-17-0.8944099378881988,0.7,265-32-0.8922558922558923,2000.0,0.272727,setup1_legal,0.600000,0.176471,0.581291
5,0.900621,None,297.0,144-17-0.8944099378881988,0.8,265-32-0.8922558922558923,2000.0,0.272727,setup1_legal,0.600000,0.176471,0.581291
6,0.770186,None,297.0,73-88-0.546583850931677,None,136-161-0.5420875420875421,0.0,0.795580,setup1_company_or_business_related,0.774194,0.818182,0.765255
7,0.745342,None,297.0,73-88-0.546583850931677,0.4,136-161-0.5420875420875421,2000.0,0.773481,setup1_company_or_business_related,0.752688,0.795455,0.740193
8,0.745342,None,297.0,73-88-0.546583850931677,0.5,136-161-0.5420875420875421,2000.0,0.773481,setup1_company_or_business_related,0.752688,0.795455,0.740193
9,0.739130,None,297.0,73-88-0.546583850931677,0.6,136-161-0.5420875420875421,2000.0,0.774194,setup1_company_or_business_related,0.734694,0.818182,0.731009


In [ ]:
df_sgd_en.to_excel('enron_sgd_with_roc.xlsx')

# Neural Network

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

#SGD
def eval_and_print_metrics_df(clf, X_train, y_train, X_test, y_test, thresh = None, kbest = None, label = None):

    dict1 = {}


    print("Number of training samples:", len(X_train))
    print("Unlabeled samples in training set:",
          sum(1 for x in y_train if x == -1))
    
    dict1['Labeled'] = len(X_train) - sum(1 for x in y_train if x == -1)
    dict1['UnLabeled'] = sum(1 for x in y_train if x == -1)
    dict1['label_used'] = label

    tr_0, tr_1 = sum(1 for x in y_train if x == 0), sum(1 for x in y_train if x == 1)
    te_0, te_1 = sum(1 for x in y_test if x == 0), sum(1 for x in y_test if x == 1)

    dict1['Train 0-1-trivial'] = str(tr_0)+'-'+str(tr_1)+'-'+\
                                                str(max(tr_0, tr_1)/(tr_0+tr_1))

    dict1['Test 0-1-trivial'] = str(te_0)+'-'+str(te_1)+'-'+\
                                                str(max(te_0, te_1)/(te_0+te_1))

    
    # if sum(1 for x in y_train if x == -1) == 0:
    #     dict1['type'] = 'Supervised'
    # else:
    #     dict1['type'] = 'Semi-Supervised'

    dict1['Threshold'] = thresh
    dict1['K-Best'] = kbest

    # print('y_train:', y_train)
    with tf.device('/device:GPU:0'):
      clf.fit(X_train, y_train)
      y_pred = np.array(clf.predict(X_test))
      
    print("Micro-averaged F1 score on test set: "
          "%0.3f" % f1_score(y_test, y_pred, average='micro'))
    print("Accuracy Score: ", accuracy_score(y_test, y_pred))

    dict1['Accuracy'] = accuracy_score(y_test, y_pred)
    dict1['roc-auc'] = roc_auc_score(y_test, y_pred)
    dict1['precision'] = precision_score(y_test, y_pred)
    dict1['recall'] = recall_score(y_test, y_pred)
    dict1['f1'] = f1_score(y_test, y_pred)

    print("-" * 10)
    print()

    return dict1

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier

tf.random.set_seed(0)
def create_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(100, activation='relu'),
      tf.keras.layers.Dense(50),
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                # loss='categorical_crossentropy',
                metrics=['accuracy'])
  
  return model



model = KerasClassifier(build_fn=create_model, epochs=20, verbose=0)

In [ ]:
mlp_params = dict(hidden_layer_sizes=(100,50), max_iter=50,activation = 'relu',solver='adam',random_state=1)
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8, max_features=30000)


if __name__ == "__main__":

    df_mlp_en = pd.DataFrame()

    # kbest_list=[4, 5, 6, 7, 8]
    threshold=[0.4, 0.5, 0.6, 0.7, 0.8]
    for i in setup_1 + setup_2: 
      tf.keras.backend.clear_session()     
      j = i[:-4]
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Enron DATA - ", j, "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

      X_train, X_test, y_train, y_test = pd.read_csv(os.path.join(path, j+'_train.csv'))[main_labels],\
            pd.read_csv(os.path.join(path, j+'_test.csv'))[main_labels], pd.read_csv(os.path.join(path, j+'_train.csv'))[j],\
            pd.read_csv(os.path.join(path, j+'_test.csv'))[j]

      
      X_u = pd.read_csv(os.path.join(path, 'enron_unlabeled_2k.csv'))
      y_u = pd.Series(np.array([-1 for i in range(X_u.shape[0])]))


      to_be_used = 'full_message'
      X_train = X_train[to_be_used]
      X_test = X_test[to_be_used]
      X_u = X_u[to_be_used]



      print("Supervised SGDClassifier on the training data:")
      # Supervised Pipeline
      pipeline = Pipeline([
          ('vect', CountVectorizer(**vectorizer_params)),
          ('tfidf', TfidfTransformer()), 
          # ('kbest', MySelectKBest(f_classif, k=20000)),         
          ('dense', FunctionTransformer(lambda x: x.toarray(), accept_sparse=True)),
          ('clf', model)
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_train, y_train, X_test, y_test, thresh = None, kbest = None, label = j)
      df_mlp_en = df_mlp_en.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on the training data (rest "
            "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
        tf.keras.backend.clear_session()
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            # ('kbest', MySelectKBest(f_classif, k=20000)),
            ('dense', FunctionTransformer(lambda x: x.toarray(), accept_sparse=True)),
            ('clf', SelfTrainingClassifier(model, criterion = 'threshold', threshold = t, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, pd.concat([X_train, X_u]), pd.concat([y_train, y_u]), X_test, y_test, thresh = t, kbest = None, label=j)
        df_mlp_en = df_mlp_en.append(temp, ignore_index=True)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Enron DATA -  setup1_legal ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised SGDClassifier on the training data:
Number of training samples: 297
Unlabeled samples in training set: 0


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Micro-averaged F1 score on test set: 0.901
Accuracy Score:  0.9006211180124224
----------

SelfTrainingClassifier on the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 2297
Unlabeled samples in training set: 2000
End of iteration 1, added 2000 new labels.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Micro-averaged F1 score on test set: 0.901
Accuracy Score:  0.9006211180124224
----------

---------------------------------Threshold =  0.5 ---------------------------------
Number of training samples: 2297
Unlabeled samples in training set: 2000
End of iteration 1, added 2000 new labels.


...

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Micro-averaged F1 score on test set: 0.876
Accuracy Score:  0.8757763975155279
----------

---------------------------------Threshold =  0.7 ---------------------------------
Number of training samples: 2297
Unlabeled samples in training set: 2000
End of iteration 1, added 2000 new labels.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Micro-averaged F1 score on test set: 0.882
Accuracy Score:  0.8819875776397516
----------

---------------------------------Threshold =  0.8 ---------------------------------
Number of training samples: 2297
Unlabeled samples in training set: 2000
End of iteration 1, added 2000 new labels.
Micro-averaged F1 score on test set: 0.876
Accuracy Score:  0.8757763975155279
----------



/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
df_mlp_en

,Accuracy,K-Best,Labeled,Test 0-1-trivial,Threshold,Train 0-1-trivial,UnLabeled,f1,label_used,precision,recall,roc-auc
0,0.900621,None,297.0,144-17-0.8944099378881988,None,265-32-0.8922558922558923,0.0,0.272727,setup1_legal,0.600000,0.176471,0.581291
1,0.900621,None,297.0,144-17-0.8944099378881988,0.4,265-32-0.8922558922558923,2000.0,0.272727,setup1_legal,0.600000,0.176471,0.581291
2,0.900621,None,297.0,144-17-0.8944099378881988,0.5,265-32-0.8922558922558923,2000.0,0.272727,setup1_legal,0.600000,0.176471,0.581291
3,0.900621,None,297.0,144-17-0.8944099378881988,0.6,265-32-0.8922558922558923,2000.0,0.272727,setup1_legal,0.600000,0.176471,0.581291
4,0.900621,None,297.0,144-17-0.8944099378881988,0.7,265-32-0.8922558922558923,2000.0,0.272727,setup1_legal,0.600000,0.176471,0.581291
5,0.900621,None,297.0,144-17-0.8944099378881988,0.8,265-32-0.8922558922558923,2000.0,0.272727,setup1_legal,0.600000,0.176471,0.581291
6,0.776398,None,297.0,73-88-0.546583850931677,None,136-161-0.5420875420875421,0.0,0.802198,setup1_company_or_business_related,0.776596,0.829545,0.770937
7,0.770186,None,297.0,73-88-0.546583850931677,0.4,136-161-0.5420875420875421,2000.0,0.810256,setup1_company_or_business_related,0.738318,0.897727,0.757083
8,0.770186,None,297.0,73-88-0.546583850931677,0.5,136-161-0.5420875420875421,2000.0,0.808290,setup1_company_or_business_related,0.742857,0.886364,0.758250
9,0.776398,None,297.0,73-88-0.546583850931677,0.6,136-161-0.5420875420875421,2000.0,0.812500,setup1_company_or_business_related,0.750000,0.886364,0.765100


In [ ]:
df_mlp_en.to_excel('enron_mlp_with_roc.xlsx')